In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import librosa

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data exploration 

In [ ]:
train_audio_path = '/kaggle/input/birdclef-2024/train_audio'

data = []

for bird_species in os.listdir(train_audio_path) :
    species_folder = os.path.join(train_audio_path, bird_species)
    
    if os.path.isdir(species_folder) :
        for audio_file in os.listdir(species_folder) :
            if audio_file.endswith('.ogg') :
                audio_path = os.path.join(species_folder, audio_file)
                
                signal, sr = librosa.load(audio_path, sr= None)
                
                mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
                mfccs_mean = np.mean(mfccs.T, axis=0)
                
                data.append(
                {
                    'species': bird_species,
                    'file': audio_file,
                    'mfcc': mfccs_mean
                }
                )
                
df = pd.DataFrame(data)

print(df.head())

# Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

y = df['species']
X = np.vstack(df['mfcc'].values)

# Diviser en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner un classificateur (par exemple, Random Forest)
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# Prédictions et évaluation
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Précision du modèle : {accuracy * 100:.2f}%")